### 1. Carga de datos y preprocesamiento

Se trabajó con el dataset oficial de la campaña de marketing de una entidad bancaria en Portugal, cuyo objetivo era predecir si un cliente aceptaría o no una oferta de depósito a plazo.

Se identificaron 10 variables numéricas y 10 categóricas.  
Se aplicó un preprocesamiento profesional basado en:

- Escalado de variables numéricas con `StandardScaler`
- Codificación de variables categóricas con `OneHotEncoder`
- Eliminación de la variable `duration`, ya que su uso implica fuga de información
- Conversión de la variable objetivo `y` a valores binarios: `no → 0`, `yes → 1`

También se generó una versión del dataset con todas las columnas procesadas y otra versión reducida con solo 3 variables seleccionadas para análisis y producción.

📌 **Importante**: la variable objetivo está **fuertemente desbalanceada**, con menos del 12% de valores positivos (`y = 1`). Por este motivo, se usaron métricas adecuadas y estrategias de balanceo en los modelos.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Cargar dataset
df = pd.read_csv('data/bank-additional-full.csv', sep=';')

# ❗ Eliminar columna 'duration' porque no puede usarse en un modelo realista
df = df.drop(columns=['duration'])

# Codificar variable objetivo (0: no, 1: yes)
df['y'] = df['y'].map({'no': 0, 'yes': 1})

# Separar features y target
X = df.drop(columns=['y'])
y = df['y']

# Detectar columnas categóricas y numéricas
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocesador: Escalar numéricas y codificar categóricas
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

### 2. Selección automática de columnas con RFECV y métrica F1

Se utilizó `RFECV` (Recursive Feature Elimination con Validación Cruzada) junto con un modelo `RandomForestClassifier` para seleccionar automáticamente las variables más relevantes para la predicción.

- La métrica utilizada fue **F1-score**, debido al desbalanceo de clases.
- El procedimiento seleccionó las siguientes 3 variables como más importantes: age euribor3m campaign




In [ ]:
from sklearn.feature_selection import RFECV

In [9]:
# Aplicamos preprocesamiento directamente a X
X_preprocessed = pd.DataFrame(preprocessor.fit_transform(X))

# Creamos modelo base
modelo_rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', random_state=42)

selector = RFECV(
    estimator=modelo_rf,
    step=1,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='f1',
    min_features_to_select=5,
    n_jobs=-1
)

print("⏳ Ejecutando RFECV sobre el dataset procesado...")
selector.fit(X_preprocessed, y)


⏳ Ejecutando RFECV sobre el dataset procesado...


RFECV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
      estimator=RandomForestClassifier(class_weight='balanced', n_jobs=-1,
                                       random_state=42),
      min_features_to_select=5, n_jobs=-1, scoring='f1')

In [ ]:
# Este método no fue seleccionado
# Seleccionamos las columnas útiles
X_selected = X_preprocessed.iloc[:, selector.support_]
print(f"✅ Columnas seleccionadas por RFECV: {X_selected.shape[1]} de {X_preprocessed.shape[1]}")

✅ Columnas seleccionadas por RFECV: 55 de 62


In [11]:
from sklearn.ensemble import RandomForestClassifier

# Aplicar preprocesamiento completo
X_processed = preprocessor.fit_transform(X)

# Entrenar RandomForest completo
modelo_base = RandomForestClassifier(n_jobs=-1, class_weight='balanced', random_state=42)
modelo_base.fit(X_processed, y)

# Obtener nombres de columnas (incluyendo one-hot codificadas)
cat_ohe = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols)
column_names = np.concatenate([num_cols, cat_ohe])

# Importancia de variables
importancias = modelo_base.feature_importances_
importancia_df = pd.Series(importancias, index=column_names).sort_values(ascending=False)


In [12]:
# Elegimos top N (ajustá acá)
top_n = 3
top_features = importancia_df.head(top_n)
print("✅ Top variables seleccionadas:")
print(top_features)


✅ Top variables seleccionadas:
age          0.140706
euribor3m    0.129510
campaign     0.078772
dtype: float64


### 3.1 Entrenamiento de Logistic Regression con las 3 variables seleccionadas (ya escaladas)

Modelo base y simple, entrenado con solo 3 variables seleccionadas automáticamente (`age`, `euribor3m`, `campaign`) tras escalado con `StandardScaler`.

Se utilizó `class_weight='balanced'` para compensar el desbalanceo de clases, y se evaluó usando validación cruzada.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.96    | 0.25    |
| Recall             | 0.73    | 0.73    |
| F1-score           | 0.83    | 0.38    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.73  
- **ROC AUC**: 0.7615

### 📊 Matriz de confusión:

[[5317 1993] [ 248 680]] = [[ TN FP] [ FN TP ]]

### ✅ Conclusión:
Buen recall en la clase 1 (clientes que sí compraron), pero con muy baja precisión. Esto indica muchos falsos positivos, aunque identifica bastantes positivos reales.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [14]:
# Filtrar columnas top del dataset procesado
top_vars = top_features.index.tolist()
X_df_processed = pd.DataFrame(X_processed, columns=column_names)
X_top = X_df_processed[top_vars]

# Dividir en train/test (estratificado)
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, stratify=y, random_state=42)

# Definir el modelo
lr = LogisticRegression(class_weight='balanced', max_iter=1000)

# Hiperparámetros a buscar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}

# Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
grid_lr = GridSearchCV(lr, param_grid, scoring='f1', cv=cv, n_jobs=-1)
grid_lr.fit(X_train, y_train)

# Mejor modelo encontrado
best_lr = grid_lr.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros: {grid_lr.best_params_}")

✅ Mejor combinación de hiperparámetros: {'C': 0.01, 'solver': 'liblinear'}


In [15]:
# Predicciones
y_pred = best_lr.predict(X_test)
y_prob = best_lr.predict_proba(X_test)[:, 1]

# Evaluación
print("\n🔍 Evaluación: Logistic Regression (top 3 variables)")
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.4f}")


🔍 Evaluación: Logistic Regression (top 3 variables)
              precision    recall  f1-score   support

           0       0.96      0.73      0.83      7310
           1       0.25      0.73      0.38       928

    accuracy                           0.73      8238
   macro avg       0.60      0.73      0.60      8238
weighted avg       0.88      0.73      0.78      8238

Matriz de confusión:
[[5317 1993]
 [ 248  680]]
ROC AUC: 0.7615


### 3.2 Entrenamiento de Logistic Regression con todas las columnas.

Modelo entrenado con todo el dataset preprocesado (numerical + categóricas codificadas).  
Se optimizó con `GridSearchCV`, siendo los mejores hiperparámetros: `C=10`, `solver='liblinear'`.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.95    | 0.37    |
| Recall             | 0.86    | 0.65    |
| F1-score           | 0.90    | 0.47    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.84  
- **ROC AUC**: 0.8009

### 📊 Matriz de confusión:

[[6282 1028] [ 329 599]] = [[ TN FP] [ FN TP ]]


### ✅ Conclusión:
Mejora significativa respecto al modelo con solo 3 variables.  
Sube el F1-score en clase 1 a 0.47 con buena precisión (0.37) y excelente balance general.

In [16]:
# Usamos el dataset completo procesado (ya escalado y one-hot aplicado)
X_full = pd.DataFrame(X_processed, columns=column_names)

# División train/test
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y, test_size=0.2, stratify=y, random_state=42)

# Modelo base
lr_full = LogisticRegression(class_weight='balanced', max_iter=1000)

# Hiperparámetros
param_grid_lr_full = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}

# GridSearchCV
grid_lr_full = GridSearchCV(lr_full, param_grid_lr_full, scoring='f1', cv=cv, n_jobs=-1)
grid_lr_full.fit(X_train_full, y_train_full)

# Mejor modelo
best_lr_full = grid_lr_full.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros (LR todas): {grid_lr_full.best_params_}")

# Predicción
y_pred_lr_full = best_lr_full.predict(X_test_full)
y_prob_lr_full = best_lr_full.predict_proba(X_test_full)[:, 1]

# Evaluación
print("\n📊 Logistic Regression (todas las columnas)")
print(classification_report(y_test_full, y_pred_lr_full))
print("Matriz de confusión:")
print(confusion_matrix(y_test_full, y_pred_lr_full))
print(f"ROC AUC: {roc_auc_score(y_test_full, y_prob_lr_full):.4f}")

✅ Mejor combinación de hiperparámetros (LR todas): {'C': 10, 'solver': 'liblinear'}

📊 Logistic Regression (todas las columnas)
              precision    recall  f1-score   support

           0       0.95      0.86      0.90      7310
           1       0.37      0.65      0.47       928

    accuracy                           0.84      8238
   macro avg       0.66      0.75      0.69      8238
weighted avg       0.88      0.84      0.85      8238

Matriz de confusión:
[[6282 1028]
 [ 329  599]]
ROC AUC: 0.8009


### 4.1 Entrenamiento de Random Forest con 3 columnas seleccionadas

Se entrenó un RandomForestClassifier con solo las 3 variables seleccionadas.  
Se ajustaron hiperparámetros y se aplicó validación cruzada con F1 como métrica principal.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.95    | 0.38    |
| Recall             | 0.87    | 0.64    |
| F1-score           | 0.91    | 0.47    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.84  
- **ROC AUC**: 0.7881

### 📊 Matriz de confusión:

[[6331 979] [ 336 592]]  =  [[ TN FP] [ FN TP ]]

### ✅ Conclusión:
Muy buen recall (0.64) en clase 1, y F1 más alto que Logistic base.  
Modelo balanceado, sin perder simplicidad.

In [17]:
from sklearn.ensemble import RandomForestClassifier

# Dataset ya escalado y con columnas filtradas
X_rf = X_df_processed[top_vars]

# Dividir en train/test
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y, test_size=0.2, stratify=y, random_state=42)

# Modelo base
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)

# Hiperparámetros a buscar
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

# Grid Search
grid_rf = GridSearchCV(rf, param_grid_rf, scoring='f1', cv=cv, n_jobs=-1)
grid_rf.fit(X_train_rf, y_train_rf)

# Mejor modelo
best_rf = grid_rf.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros (Random Forest): {grid_rf.best_params_}")

# Predicciones
y_pred_rf = best_rf.predict(X_test_rf)
y_prob_rf = best_rf.predict_proba(X_test_rf)[:, 1]

# Evaluación
print("\n🔍 Evaluación: Random Forest (top 3 variables)")
print(classification_report(y_test_rf, y_pred_rf))
print("Matriz de confusión:")
print(confusion_matrix(y_test_rf, y_pred_rf))
print(f"ROC AUC: {roc_auc_score(y_test_rf, y_prob_rf):.4f}")

✅ Mejor combinación de hiperparámetros (Random Forest): {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}

🔍 Evaluación: Random Forest (top 3 variables)
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      7310
           1       0.38      0.64      0.47       928

    accuracy                           0.84      8238
   macro avg       0.66      0.75      0.69      8238
weighted avg       0.89      0.84      0.86      8238

Matriz de confusión:
[[6331  979]
 [ 336  592]]
ROC AUC: 0.7881


### 4.2 Entrenamiento de Random Forest con todas las columnas

Modelo entrenado con todas las variables disponibles (62 features post codificación).  
Mejor rendimiento en recall y F1 respecto al modelo reducido.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.95    | 0.42    |
| Recall             | 0.89    | 0.63    |
| F1-score           | 0.92    | 0.50    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.86  
- **ROC AUC**: 0.8128

### 📊 Matriz de confusión:

[[6503 807] [ 342 586]] = [[ TN FP] [ FN TP ]]


### ✅ Conclusión:
Excelente desempeño global y mejora en clase 1, logrando un buen equilibrio entre precisión y recall.


In [18]:
# Dataset completo procesado
X_rf_full = pd.DataFrame(X_processed, columns=column_names)

# División en train/test
X_train_rf_full, X_test_rf_full, y_train_rf_full, y_test_rf_full = train_test_split(
    X_rf_full, y, test_size=0.2, stratify=y, random_state=42
)

# Definir modelo base
rf_full = RandomForestClassifier(n_jobs=-1, class_weight='balanced', random_state=42)

# Hiperparámetros a ajustar
param_grid_rf_full = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

# GridSearchCV
grid_rf_full = GridSearchCV(rf_full, param_grid_rf_full, scoring='f1', cv=cv, n_jobs=-1)
grid_rf_full.fit(X_train_rf_full, y_train_rf_full)

# Mejor modelo
best_rf_full = grid_rf_full.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros (Random Forest todas): {grid_rf_full.best_params_}")

# Predicciones
y_pred_rf_full = best_rf_full.predict(X_test_rf_full)
y_prob_rf_full = best_rf_full.predict_proba(X_test_rf_full)[:, 1]

# Evaluación
print("\n📊 Random Forest (todas las columnas)")
print(classification_report(y_test_rf_full, y_pred_rf_full))
print("Matriz de confusión:")
print(confusion_matrix(y_test_rf_full, y_pred_rf_full))
print(f"ROC AUC: {roc_auc_score(y_test_rf_full, y_prob_rf_full):.4f}")


✅ Mejor combinación de hiperparámetros (Random Forest todas): {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}

📊 Random Forest (todas las columnas)
              precision    recall  f1-score   support

           0       0.95      0.89      0.92      7310
           1       0.42      0.63      0.50       928

    accuracy                           0.86      8238
   macro avg       0.69      0.76      0.71      8238
weighted avg       0.89      0.86      0.87      8238

Matriz de confusión:
[[6503  807]
 [ 342  586]]
ROC AUC: 0.8128


### 5.1 Entrenamiento de XGBoost con 3 columnas seleccionadas.

Modelo potente aún con solo 3 columnas.  
Se ajustaron hiperparámetros (`scale_pos_weight`, `n_estimators`, etc.) y se obtuvo un resultado competitivo.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.95    | 0.44    |
| Recall             | 0.90    | 0.59    |
| F1-score           | 0.92    | 0.50    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.87  
- **ROC AUC**: 0.7947

### 📊 Matriz de confusión:

[[6605 705] [ 384 544]] = [[ TN FP] [ FN TP ]]

### ✅ Conclusión:

Muy competitivo para solo 3 variables.  
Mejor F1 que Logistic y Random Forest en formato reducido.

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
# Filtrar columnas top
X_xgb = X_df_processed[top_vars]

# División train/test
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb, y, test_size=0.2, stratify=y, random_state=42
)

# Modelo base
xgb = XGBClassifier(
    eval_metric='logloss',
    objective='binary:logistic',
    random_state=42
)

# Hiperparámetros + scale_pos_weight
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'scale_pos_weight': [3, 5, 10]  # importante para desbalanceo
}

# GridSearchCV
grid_xgb = GridSearchCV(xgb, param_grid_xgb, scoring='f1', cv=cv, n_jobs=-1)
grid_xgb.fit(X_train_xgb, y_train_xgb)

# Mejor modelo
best_xgb = grid_xgb.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros (XGBoost top 3): {grid_xgb.best_params_}")


✅ Mejor combinación de hiperparámetros (XGBoost top 3): {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'scale_pos_weight': 5}


In [21]:
# Predicciones
y_pred_xgb = best_xgb.predict(X_test_xgb)
y_prob_xgb = best_xgb.predict_proba(X_test_xgb)[:, 1]

# Evaluación
print("\n📊 XGBoost (top 3 variables)")
print(classification_report(y_test_xgb, y_pred_xgb))
print("Matriz de confusión:")
print(confusion_matrix(y_test_xgb, y_pred_xgb))
print(f"ROC AUC: {roc_auc_score(y_test_xgb, y_prob_xgb):.4f}")


📊 XGBoost (top 3 variables)
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      7310
           1       0.44      0.59      0.50       928

    accuracy                           0.87      8238
   macro avg       0.69      0.74      0.71      8238
weighted avg       0.89      0.87      0.88      8238

Matriz de confusión:
[[6605  705]
 [ 384  544]]
ROC AUC: 0.7947


### 5.2 Entrenamiento de XGBoost con todas las columnas.

Fue el modelo con **mejor rendimiento absoluto**.  
Se entrenó sobre todas las columnas del dataset y se ajustó con GridSearchCV.

### 🔍 Métricas obtenidas:

| Métrica            | Clase 0 | Clase 1 |
|--------------------|---------|---------|
| Precision          | 0.95    | 0.48    |
| Recall             | 0.92    | 0.59    |
| F1-score           | 0.93    | 0.53    |
| Soporte            | 7310    | 928     |

- **Accuracy**: 0.88  
- **ROC AUC**: 0.8181

### 📊 Matriz de confusión:

[[6703 607] [ 377 551]] = [[ TN FP] [ FN TP ]]

### ✅ Conclusión:
Mejor F1 en clase 1 de todos los modelos.  
Excelente rendimiento, buena precisión y cobertura.  
Este es el **modelo recomendado** si el objetivo es maximizar la predicción de suscriptores reales (`y=1`).

In [22]:
# Dataset completo procesado (ya escalado y codificado)
X_xgb_full = pd.DataFrame(X_processed, columns=column_names)

# División train/test
X_train_xgb_full, X_test_xgb_full, y_train_xgb_full, y_test_xgb_full = train_test_split(
    X_xgb_full, y, test_size=0.2, stratify=y, random_state=42
)

# Definir modelo
xgb_full = XGBClassifier(
    eval_metric='logloss',
    objective='binary:logistic',
    random_state=42
)

# Hiperparámetros
param_grid_xgb_full = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'scale_pos_weight': [3, 5, 10]
}

# GridSearchCV
grid_xgb_full = GridSearchCV(xgb_full, param_grid_xgb_full, scoring='f1', cv=cv, n_jobs=-1)
grid_xgb_full.fit(X_train_xgb_full, y_train_xgb_full)

# Mejor modelo
best_xgb_full = grid_xgb_full.best_estimator_
print(f"✅ Mejor combinación de hiperparámetros (XGBoost full): {grid_xgb_full.best_params_}")


✅ Mejor combinación de hiperparámetros (XGBoost full): {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 5}


In [23]:
# Predicciones
y_pred_xgb_full = best_xgb_full.predict(X_test_xgb_full)
y_prob_xgb_full = best_xgb_full.predict_proba(X_test_xgb_full)[:, 1]

# Evaluación
print("\n📊 XGBoost (todas las columnas)")
print(classification_report(y_test_xgb_full, y_pred_xgb_full))
print("Matriz de confusión:")
print(confusion_matrix(y_test_xgb_full, y_pred_xgb_full))
print(f"ROC AUC: {roc_auc_score(y_test_xgb_full, y_prob_xgb_full):.4f}")


📊 XGBoost (todas las columnas)
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      7310
           1       0.48      0.59      0.53       928

    accuracy                           0.88      8238
   macro avg       0.71      0.76      0.73      8238
weighted avg       0.89      0.88      0.89      8238

Matriz de confusión:
[[6703  607]
 [ 377  551]]
ROC AUC: 0.8181


### 6. Conclusión y elección del modelo final

Luego de comparar todos los modelos entrenados, tanto en versión reducida como completa, se concluye:

✅ **XGBoost con todas las columnas** es el mejor modelo para este problema:
- Mayor F1-score en clase positiva (`y=1`): **0.53**
- Mayor ROC AUC: **0.8181**
- Balance sólido entre precisión y recall

📌 Si el objetivo es identificar correctamente a los clientes que van a suscribirse, **este modelo ofrece la mejor cobertura y rendimiento global**.

📦 También se generó un modelo simplificado con solo 3 variables (`age`, `euribor3m`, `campaign`), el cual mantiene un rendimiento competitivo, siendo ideal para producción o aplicaciones en tiempo real.



## 📊 Cuadro comparativo final de métricas

| Modelo           | Dataset     | Accuracy | Precision(1) | Recall(1) | F1(1) | ROC AUC |
|------------------|-------------|----------|--------------|-----------|-------|---------|
| Logistic Reg     | 3 col       | 0.73     | 0.25         | 0.73      | 0.38  | 0.7615  |
| Logistic Reg     | Full        | 0.84     | 0.37         | 0.65      | 0.47  | 0.8009  |
| Random Forest    | 3 col       | 0.84     | 0.38         | 0.64      | 0.47  | 0.7881  |
| Random Forest    | Full        | 0.86     | 0.42         | 0.63      | 0.50  | 0.8128  |
| XGBoost          | 3 col       | 0.87     | 0.44         | 0.59      | 0.50  | 0.7947  |
| **XGBoost**      | **Full**    | **0.88** | **0.48**     | **0.59**  | **0.53**| **0.8181** |



### 7. Creación del modelo de producción

📌 Elección:
🔹 XGBoost con 3 columnas ✅

- Ofrece muy buen rendimiento (F1 = 0.50), mejor que los otros modelos con 3 columnas, y es muy ágil. "Pesa" menos de 600Kb.

- ¿Por qué no Logistic con 3 columnas?
  - Su precisión en clase 1 es solo 0.25.
  - Tiene mucho más falsos positivos.
  - Peor F1 (0.38 vs. 0.50)

✅ Conclusión:
Para un modelo ágil, preciso, fácil de usar y rápido para predicción en tiempo real en Dash, el ideal es:
XGBoost entrenado con 3 columnas: age, euribor3m, campaign

In [26]:
import pandas as pd
from xgboost import XGBClassifier
import joblib

# Dataset con solo 3 columnas seleccionadas + 'y'
df_simple = df[['age', 'euribor3m', 'campaign', 'y']].copy()

# Separar X e y
X = df_simple.drop(columns='y')
y = df_simple['y']

# Entrenar modelo XGBoost
modelo_final = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.01,
    scale_pos_weight=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

modelo_final.fit(X, y)

# Guardar modelo entrenado para usar en producción (Dash)
joblib.dump(modelo_final, 'modelo_xgboost_3cols.pkl')

c:\Users\j-jai\Desktop\GST\gstvenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:01:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


['modelo_xgboost_3cols.pkl']